In [ ]:
# This notebook places the wartebox (a planar water "slab") on top of SAMs

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pylab as pl

In [2]:
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

def linecount(file_name):
    with open(file_name, 'r') as foo:
        x = len(foo.readlines())
        return x

In [3]:
# Set system names and path to directory with .pdb files
pc=[0,11,22,33,44,50,37]

# Single system for testing
#pc=[11]

samsfolder = "/Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/SAM_startfiles_v2_12x12/"

os.chdir(samsfolder)

In [8]:
# Use forward slash '/' at the end of the paths!!!

#folder with water and SAM .gro files
workingdir = samsfolder

# folder for storing shifted sams: can be deleted if everything works fine
unwantedfolder = workingdir + "delete/" #path unwanted files

# if folder does not exists, create it
if not os.path.isdir(unwantedfolder):
    os.mkdir(unwantedfolder)

In [9]:
# Print SAMs box sizes (last line of .gro file)
for m in pc:
    newfile = 'start%d.gro'%(m, )   
    last_line = open(newfile, "r").readlines()[-1]
    Line_len = len(last_line)
    print newfile, ":", last_line

start0.gro :    6.00000   5.19620   1.52200

start11.gro :    6.00000   5.19620   1.52200

start22.gro :    6.00000   5.19620   1.52200

start33.gro :    6.00000   5.19620   1.52200

start44.gro :    6.00000   5.19620   1.52200

start50.gro :    6.00000   5.19620   1.52200

start37.gro :    6.00000   5.19620   1.52200



In [10]:
# sam sizes in z-direction (not box size!!)
sam={}

#sizes of simulation box of sams
xbox={}
ybox={}
zbox={}

mynumbers = []
mynumbers.append([float(n) for n in last_line.split()])

for i in pc:
    sam[i]= mynumbers[0][2]

    xbox[i]=mynumbers[0][0]
    ybox[i]=mynumbers[0][1]
    zbox[i]=mynumbers[0][2]

# water drop size in z-direction (they could also be read, look at script ExtendSam.ipynb)
waterheight = 1.51896 # last line of NPT_water.gro

# New box size in z-direction
zboxnew = waterheight + zbox[i]

print "zboxnew =", zboxnew
print sam
print xbox, ybox, zbox

zboxnew = 3.04096
{0: 1.522, 33: 1.522, 37: 1.522, 11: 1.522, 44: 1.522, 50: 1.522, 22: 1.522}
{0: 6.0, 33: 6.0, 37: 6.0, 11: 6.0, 44: 6.0, 50: 6.0, 22: 6.0} {0: 5.1962, 33: 5.1962, 37: 5.1962, 11: 5.1962, 44: 5.1962, 50: 5.1962, 22: 5.1962} {0: 1.522, 33: 1.522, 37: 1.522, 11: 1.522, 44: 1.522, 50: 1.522, 22: 1.522}


In [11]:
os.chdir(workingdir)
    
for i in pc:   
    
    # Initial .gro file names
    startsamfile='start%d.gro'%(i, )
    startwaterfile='NPT_water.gro'
    
    # Temporary file names
    samfile='start'+str(i)+'_c.gro'
    waterfile='NPT_water_c.gro'
    
    # New file names
    newsystfile='NPT_sam'+str(i)+'_water_c.gro'
    endfile='NPT_sam'+str(i)+'_water.gro'
    
    # Center SAMs in box
    os.system("gmx editconf -f %s -o %s -c" %(startsamfile, samfile, ))     
               
    # Set water box size equal to SAM box size
    os.system("gmx editconf -f %s -o %s -box %5.3f %5.3f %5.3f" %(startwaterfile, waterfile, xbox[i], ybox[i], zbox[i], ))
               
    # Move water drops up 
    os.system("gmx editconf -f %s -o %s -translate 0 0 %5.3f" %(waterfile, waterfile, (zbox[i]/2) +(waterheight/2), )) 
        
##### START THE MERGING PROCESS OF water drops + SAMs #################

    # Create the new file by openning and closing it
    creating = open(newsystfile, 'w+')
    creating.close()
        
    #linecount
    linewater=linecount(waterfile)
    linesam=linecount(samfile)
        
    lastline = open(samfile, "r").readlines()[-1]
    with open(newsystfile, 'a') as f1:
        with open(samfile, 'r') as g:
            l = 0
            for line in g:
                if l < linesam-1: 
                    f1.write(line)
                l = l+1   
        with open(waterfile, 'r') as h:
            l = 0
            for line in h:                        
                if l > 1 and l < linewater-1:
                    f1.write(line)
                l = l + 1
        f1.write(lastline)

             
    # Replace atom number
    atomssam = linesam-3
    atomswater = linewater-3
            
    replace_line(newsystfile, 1, str(atomssam+atomswater) + "\n")
        
    # Replace first line (title)
    replace_line(newsystfile, 0, "SAM "+str(i)+"% OH-coverage + with planar water slab" + "\n")
        
    # Change box height (z-direction)
    os.system("gmx editconf -f %s -o %s -box %5.3f %5.3f %5.3f" %(newsystfile, endfile, xbox[i], ybox[i], zboxnew, ))
        
    # Translate system to bottom of box
    #os.system("gmx editconf -f %s -o %s -translate %8.5f %8.5f %8.5f" %(endfile, endfile, 0, 0,(zbox[i]-water[j]+0.5)/2, ))

In [12]:
# read position of bottom atom of SAM and save in dictionary zshift and translate system
line_num = 13 # some chains are longer then others: 12 or 13. Tested with "if-statement"
zshift = {}
for i in pc:
    endname='NPT_sam'+str(i)+'_water.gro'
    lines = open(endname, 'r').readlines()
    myline = lines[line_num]
    mynumbers = []
    mynumbers.append([str(n) for n in myline.split()])
    firstchain = str(mynumbers[0][0])
    #print len(firstchain), firstchain
    if firstchain == '1SAM' or firstchain == '1XOH':
        print 'chainlength = ', line_num
        print myline
        zshift[i] = float(mynumbers[0][5])
        print firstchain, zshift[i]
        os.system("gmx editconf -f %s -o %s -translate %8.5f %8.5f %8.5f" %(endname, endname, 0, 0,-zshift[i], ))
    else:
        print 'chainlength = ', line_num
        line_num = 12
        myline = lines[line_num]
        print myline
        mynumbers = []
        mynumbers.append([str(n) for n in myline.split()])
        zshift[i] = float(mynumbers[0][5])
        print str(mynumbers[0][0]), zshift[i]
        os.system("gmx editconf -f %s -o %s -translate %8.5f %8.5f %8.5f" %(endname, endname, 0, 0,-zshift[i], ))

chainlength =  13
    1SAM    CAB   11   0.172   0.534  -0.157  0.0000  0.0000  0.0000

1SAM -0.157
chainlength =  12
    1SAM    CAB   11   0.175   0.538  -0.160  0.0000  0.0000  0.0000

1SAM -0.16
chainlength =  12
    1XOH    CAD   11   0.087   0.477  -0.054  0.0000  0.0000  0.0000

1XOH -0.054
chainlength =  12
    1SAM    CAB   11   0.173   0.546  -0.166  0.0000  0.0000  0.0000

1SAM -0.166
chainlength =  12
    1XOH    CAD   11   0.088   0.476  -0.059  0.0000  0.0000  0.0000

1XOH -0.059
chainlength =  12
    1XOH    CAD   11   0.083   0.483  -0.061  0.0000  0.0000  0.0000

1XOH -0.061
chainlength =  12
    1SAM    CAB   11   0.170   0.546  -0.167  0.0000  0.0000  0.0000

1SAM -0.167


In [13]:
#Renaming and moving endfiles to endfolder

startfolder = workingdir
endfolder = workingdir + "NewVersion_v2/"

#os.system("mkdir %s"% (endfolder))
os.chdir(endfolder)

for i in pc:
    startname='NPT_sam'+str(i)+'_water.gro'
    endname='sam'+str(i)+'_water_ptensor_v2.gro'
    os.system("mv %s  %s"% (startfolder+startname,endfolder+endname, ))

In [14]:
# Remove gromacs file copies from endfolder
os.chdir(endfolder)
os.system("rm \#*")

0

In [15]:
# Delete/move not needed files from workingdir

os.chdir(workingdir)

# Remove gromacs file copies
os.system("rm \#*")

# Move to "delete" folder
for i in pc:
    
    # Temporary file names
    samfile='start'+str(i)+'_c.gro'
    waterfile='NPT_water_c.gro'
    newsystfile='NPT_sam'+str(i)+'_water_c.gro'

    os.system("mv %s  %s"%(samfile,unwantedfolder + samfile, ))
    os.system("mv %s  %s"% (waterfile,unwantedfolder + waterfile, ))
    os.system("mv %s  %s"%(newsystfile,unwantedfolder + newsystfile, ))

In [16]:
"""
For removing:
os.remove() will remove a file.

os.rmdir() will remove an empty directory.

shutil.rmtree() will delete a directory and all its contents.

For moving:
os.rename() or shutil.move()

Both employ the same syntax:
os.rename("path/to/current/file.foo", "path/to/new/desination/for/file.foo")
shutil.move("path/to/current/file.foo", "path/to/new/destination/for/file.foo")

For copying:

-------------------------------------------------------------------------
| Function          |Copies Metadata|Copies Permissions|Can Specify Buffer|
-------------------------------------------------------------------------
| shutil.copy       |      No       |        Yes       |        No        |
-------------------------------------------------------------------------
| shutil.copyfile   |      No       |         No       |        No        |
-------------------------------------------------------------------------
| shutil.copy2      |     Yes       |        Yes       |        No        |
-------------------------------------------------------------------------
| shutil.copyfileobj|      No       |         No       |       Yes        |
-------------------------------------------------------------------------

Example:
shutil.copy2('/dir/file.ext', '/new/dir')

"""

'\nFor removing:\nos.remove() will remove a file.\n\nos.rmdir() will remove an empty directory.\n\nshutil.rmtree() will delete a directory and all its contents.\n\nFor moving:\nos.rename() or shutil.move()\n\nBoth employ the same syntax:\nos.rename("path/to/current/file.foo", "path/to/new/desination/for/file.foo")\nshutil.move("path/to/current/file.foo", "path/to/new/destination/for/file.foo")\n\nFor copying:\n\n-------------------------------------------------------------------------\n| Function          |Copies Metadata|Copies Permissions|Can Specify Buffer|\n-------------------------------------------------------------------------\n| shutil.copy       |      No       |        Yes       |        No        |\n-------------------------------------------------------------------------\n| shutil.copyfile   |      No       |         No       |        No        |\n-------------------------------------------------------------------------\n| shutil.copy2      |     Yes       |        Yes     

In [17]:
import shutil
shutil.rmtree(unwantedfolder)